## EDA（探索的データ分析）
学習に使用するデータを観察しましょう

In [ ]:
import pandas as pd
import numpy as np

trn = pd.read_csv('./dataset/train_ver2.csv')
trn.head()

In [ ]:
trn.shape

In [ ]:
# forを使うと省略されないし、型も一緒に確認できる
for col in trn.columns:
    print('{}\n'.format(trn[col].head()))

In [ ]:
trn.info()

In [ ]:
num_cols = [col for col in trn.columns[:24] if trn[col].dtype in ["int64", "float64"]]
trn[num_cols].describe()

## 数値型変数に関する分析
- ncodpers：顧客固有識別番号
- ind_nuevo：最小75%が0、残りが1の新規顧客指標
- indrel：最小75%の値が1で、残りの値が99の顧客等級変数
- tipodom：標準偏差0なので全部同じ（使えない変数）
- cod_prov：1~52の値、数値型だがカテゴリ変数として意味を持つ地方コード変数
- ind_actividad_cliente：半分0、半分1の活発性指標
- renta：一家の総収入

In [ ]:
cat_cols = [col for col in trn.columns[:24] if trn[col].dtype in ["O"]]
trn[cat_cols].describe()

### カテゴリ変数についての結果値の意味
- count：該当変数の有効なデータの個数を意味する。データの個数より小さい場合は、その分欠損している。
- unique：データの固有値の個数。種類。
- top：最頻値
- freq：topの値の頻度数

In [ ]:
for col in cat_cols:
    uniq = np.unique(trn[col].astype(str));
    print("-"*50)
    print('# col {}, n_uniq {}, uniq {}'.format(col, len(uniq), uniq))

In [11]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#データの可視化
skip_cols = ['ncodpers', 'renta']
for col in trn.columns[:24]:
    # 固有値が多い特徴量は時間が掛かりすぎるのでスキップ
    if col in skip_cols:
        continue

    # 見やすくするため、領域区分と変数名を出力
    print('-'*50)
    print('col : ', col)

    # グラフの大きさ(figsize)を設定します
    f, ax = plt.subplots(figsize=(20, 9))
    # seabornを使用した棒グラフを設定します
    sns.countplot(x=col, data=trn, alpha=0.5)
    plt.show()

In [ ]:
# 月別金融商品の保有データを累積棒グラフで可視化します
months = trn['fecha_dato'].unique().tolist()
label_cols = trn.columns[24:].tolist()

label_over_time = []
for i in range(len(label_cols)):
    # 毎月、各商品の総合をgroupby().agg('sum')で計算し、label_sumに保存します
    label_sum = trn.groupby(['fecha_dato'])[label_cols[i]].agg('sum')
    label_over_time.append(label_sum.tolist())

label_sum_over_time = []
for i in range(len(label_cols)):
    # 累積棒グラフを可視化するため、n番目の商品の出力を
    # 1~n番目の商品の総合として作成
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))

# 色指定
color_list = ['#F5B7B1', '#D2B4DE', '#AED6F1', '#A2D9CE', '#ABEBC6', '#F9E79F', '#F5CBA7', '#CCD1D1']

# 図の大きさ
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    # 24個の商品についてヒストグラムを書く
    # x軸は月のデータ、y軸には累積合計、色は８色の繰り返し
    # 図のalpha値（透明度）は０．７
    sns.barplot(x=months, y=label_sum_over_time[i], color=color_list[i%8], alpha=0.7)

# 右上にLegendを追加
plt.legend([plt.Rectangle((0, 0), 1, 1, fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1, ncol=2, prop={'size':16})

In [ ]:
# 相対値を使う
label_sum_percent = (label_sum_over_time/(1.*np.asarray(label_sum_over_time).max(axis=0)))*100

# 図の大きさ
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    # 24個の商品についてヒストグラムを書く
    # x軸は月のデータ、y軸には累積合計、色は８色の繰り返し
    # 図のalpha値（透明度）は０．７
    sns.barplot(x=months, y=label_sum_percent[i], color=color_list[i%8], alpha=0.7)

# 右上にLegendを追加
plt.legend([plt.Rectangle((0, 0), 1, 1, fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1, ncol=2, prop={'size':16})

### 目的を見失わないようにする
- 今回必要な情報は顧客が新規に購買する商品
- つまり、購入しているかどうかよりは、未購入から購入した（０から１に切り替わった）点に興味がある
- そこの可視化を目指す

In [ ]:
labels = pd.read_csv('./dataset/labels.csv').astype(int)
fecha_dato = pd.read_csv('./dataset/train_ver2.csv', usecols=['fecha_dato'])

labels['date'] = fecha_dato.fecha_dato
months = np.unique(fecha_dato.fecha_dato).tolist()
label_cols = labels.columns.tolist()[:24]

label_over_time = []
for i in range(len(label_cols)):
    label_over_time.append(labels.groupby(['date'])[label_cols[i]].agg('sum').tolist())
    
label_sum_over_time = []
for i in range(len(label_cols)):
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))

# 色指定
color_list = ['#F5B7B1', '#D2B4DE', '#AED6F1', '#A2D9CE', '#ABEBC6', '#F9E79F', '#F5CBA7', '#CCD1D1']

# 図の大きさ
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.barplot(x=months, y=label_sum_over_time[i], color=color_list[i%8], alpha=0.7)

# 右上にLegendを追加
plt.legend([plt.Rectangle((0, 0), 1, 1, fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1, ncol=2, prop={'size':16})


In [ ]:
# 相対値にする
# 相対値を使う
label_sum_percent = (label_sum_over_time/(1.*np.asarray(label_sum_over_time).max(axis=0)))*100

# 図の大きさ
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.barplot(x=months, y=label_sum_percent[i], color=color_list[i%8], alpha=0.7)

# 右上にLegendを追加
plt.legend([plt.Rectangle((0, 0), 1, 1, fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1, ncol=2, prop={'size':16})